<a href="https://colab.research.google.com/github/arita37/mlmodels/blob/dev/timeseries_m4_mlmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#### Install MLMODELS in Colab
# %%capture
from time import time; t0 =time()
!  bash <(wget -qO- https://cutt.ly/mlmodels)  > log.txt
!pip install pydantic==1.4 --force
print(time() - t0, flush=True) 
import os, time
os.kill(os.getpid(), 9)

ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
--2020-06-06 17:15:12--  https://raw.githubusercontent.com/arita37/mlmodels/dev/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 441 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     441  --.-KB/s    in 0s      

2020-06-06 17:15:12 (24.6 MB/s) - ‘requirements.txt’ saved [441/441]

ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.2.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

from mlmodels import util
print(util)

<module 'mlmodels.util' from '/usr/local/lib/python3.6/dist-packages/mlmodels/util.py'>


In [3]:
# %matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm.autonotebook import tqdm
from pathlib import Path


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [4]:
"""
Using 
https://github.com/arita37/mlmodels/blob/dev/mlmodels/example/benchmark_timeseries_m4.py
 
 
"""
###############  Working directory  ######################################################
import os
dirdrive = "/content/drive/My Drive/Colab Notebooks/shared_session/timeseries_example/"

os.chdir( dirdrive )
print( os.getcwd())

/content/drive/My Drive/Colab Notebooks/shared_session/timeseries_example


In [0]:
""" naming

dir_raw :  all initial data from kaggle, ...
dir_input : all data for data_pars and model

dir_inter :  intermediate data


"""

In [0]:
!ls

In [5]:
from mlmodels.preprocess.timeseries import save_to_file

"""
from pathlib import Path
from typing import Dict, List
def save_to_file(path: Path, data: List[Dict]):
    print(f"saving time-series into {path}")
    path_dir = os.path.dirname(path)
    os.makedirs(path_dir, exist_ok=True)
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))
"""
print(save_to_file)




<function save_to_file at 0x7ff926439950>


In [6]:
"""
Manual Pre-processing

"""
import json
import pandas as pd, numpy as np


#load data  from SHARED FOLDER
data_folder = "kaggle_data/m4_dataset"


train_df   = pd.read_csv(data_folder + '/Daily-train.csv',index_col=0)
test_df    = pd.read_csv(data_folder + '/Daily-test.csv',index_col=0)
print(train_df.shape, train_df.head(5))
print(test_df.head(5))

print( train_df.columns )
print( train_df.index)



#### PARAMS ####################################################################
freq = "D"
prediction_length = 14
start_date = ""

######################## Parameters
single_pred_length = 14
submission_pred_length = single_pred_length * 2

submission=True

if submission:
    pred_length = submission_pred_length
else:
    pred_length = single_pred_length


#### Cardinality count #########################################################
cardinality_train = len(train_df) # 4227
cardinality_test  = len(test_df) # 4227
print(cardinality_train, cardinality_test)




#### output JSON Files  ########################################################
#data_folder = "kaggle_data/m4_daily_test"
gluonts_datafolder = "gluonts_data/m4_dataset"

train_file = gluonts_datafolder+"/m4_daily/train/data.json"
test_file  = gluonts_datafolder+"/m4_daily/test/data.json"
meta_file  = gluonts_datafolder+'/m4_daily/metadata.json'


#### For mlmodel data_pars
dataset_path  = Path( gluonts_datafolder+"/m4_daily" )



#### Output Metadata JSON
with open(meta_file, 'w') as f:
      f.write( json.dumps(
              {"freq": "D", 
               "prediction_length": 14, 
               "feat_static_cat": [{"name": "feat_static_cat", "cardinality":  str(cardinality_train)}]
               }
              )
          )


#### Output JSON  ##############################################################
train_target_values = [ts[~np.isnan(ts)] for ts in train_df.values]
print(train_target_values[:3])

save_to_file(train_file,
      [{ "start": "1750-01-01 00:00:00",
         "target": list(target),
         "feat_static_cat": [cat]
  }
          for cat, target in enumerate(train_target_values)
      ],
  )


#### Output JSON  ##############################################################
test_target_values = [
        np.hstack([train_ts, test_ts])
        for train_ts, test_ts in zip(train_target_values, test_df.values)
    ]
print(test_target_values[:3])

save_to_file(test_file,
    [
      {
    "start": "1750-01-01 00:00:00",
    "target": list(target),
    "feat_static_cat": [cat]
}      
        for cat, target in enumerate(test_target_values)
    ],
)



(4227, 9919)          V2       V3       V4       V5  ...  V9917  V9918  V9919  V9920
V1                                      ...                            
D1  1017.10  1019.30  1017.00  1019.20  ...    NaN    NaN    NaN    NaN
D2  2793.70  2793.80  2803.70  2805.80  ...    NaN    NaN    NaN    NaN
D3  1091.30  1088.50  1085.70  1082.90  ...    NaN    NaN    NaN    NaN
D4  1092.00  1078.00  1064.00  1050.00  ...    NaN    NaN    NaN    NaN
D5  2938.63  2956.44  2964.41  2972.41  ...    NaN    NaN    NaN    NaN

[5 rows x 9919 columns]
         V2       V3       V4      V5  ...      V12      V13      V14      V15
V1                                     ...                                    
D1  2039.20  2035.00  2051.80  2061.8  ...  2067.40  2071.40  2083.80  2080.60
D2  2986.00  3001.20  2975.90  2996.1  ...  2991.40  3024.90  3070.80  3076.90
D3  1120.70  1117.90  1115.10  1112.3  ...  1092.70  1089.90  1087.10  1084.30
D4  1190.00  1162.00  1134.00  1106.0  ...   910.00  1428.00  1

In [7]:
### Check the GluonTS Dataset
from gluonts.dataset.common import ListDataset, load_datasets

TD            = load_datasets(  metadata=dataset_path,
                train=dataset_path / "train",
                test=dataset_path / "test",)
print(TD)

TrainDatasets(metadata=MetaData(freq='D', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=14), train=<gluonts.dataset.common.FileDataset object at 0x7ff92645f160>, test=<gluonts.dataset.common.FileDataset object at 0x7ff92596d550>)


In [0]:

# MLMODELS setup
import mlmodels
from mlmodels.models import module_load
from mlmodels.util import path_norm_dict, path_norm, params_json_load
import json

from pprint import pprint as print2

json_path="m4_daily_gluonts.json"


#### Model URI and Config JSON
#model_uri   = "model_keras.gluonts_model"
#config_path = path_norm( 'model_keras/ardmn.json'  )
#config_mode = "test"  ### test/prod

model_uri="gluonts_model"

#module=None





In [0]:
#### Load JSON File (no python is included)
## DO NOT INCLUDE PYTHON variables, only string or json values
jpars = {
    "deepar": {
         "model_pars": {
             "model_uri"  : "model_gluon.gluonts_model",
             "model_name" : "deepar",
             "model_pars" : {
                 "prediction_length": 14, 
                 "freq": "D",
                 "distr_output" :  "gluonts.distribution.student_t.StudentTOutput", 
                

                
                 "use_feat_static_cat": true, 
                 "use_feat_static_real":false,
                 
                  "num_layers": 2, 
                  "num_cells": 40, 
                  "cell_type": "lstm", 
                  "dropout_rate": 0.1,

                 
                  "scaling": true, 
                  "num_parallel_samples": 100,
                  "cardinality":  [4227]
             }
             
             
             },
        "data_pars": {
            "train": true, 
            "dt_source": "",
            "data_type":"gluonts",

                
                 "use_feat_static_cat": true, 
                 "use_feat_static_real":false,

           
            "submission": false , 
            "dataset_name": "m4_daily" , 
            
            "data_path":  "gluonts_data/m4_dataset/m4_daily" , 
           
            "single_pred_length":28,
            "freq": "1D",
            "start" : "1750-01-01 00:00:00",        
            "startdate" : "1750-01-01",  
            "col_date"   : "timestamp",                
            "col_ytarget" : ["target"],
            "num_series" : 1,

            "cols_cat": [],   "cols_num" : []
                    
            },
            
            
        "compute_pars": {
            "num_samples": 100,
            
            "learning_rate"         : 1e-3,
            "epochs"                : 100,
            "num_batches_per_epoch" : 50,
            "batch_size"            : 32,   
            
            "compute_pars" : {
                
                "batch_size": 32, "clip_gradient": 100, "epochs": 100, "init": "xavier", "learning_rate": 1e-3, 
                "learning_rate_decay_factor": 0.5, 
                "hybridize": false,
                "num_batches_per_epoch": 50,
                
                "minimum_learning_rate": 5e-05, "patience": 10, "weight_decay": 1e-08
            }
        },
        
      "out_pars": {
         "path": "ztest/model_gluon/gluonts_deepar/",
         "plot_prob": true, "quantiles": [0.5]
      }
    }
}



In [9]:

#### Load JSON
jpars = json.load(open(json_path))
print2(jpars)



{'deepar': {'compute_pars': {'batch_size': 32,
                             'compute_pars': {'batch_size': 32,
                                              'clip_gradient': 100,
                                              'epochs': 100,
                                              'hybridize': False,
                                              'init': 'xavier',
                                              'learning_rate': 0.001,
                                              'learning_rate_decay_factor': 0.5,
                                              'minimum_learning_rate': 5e-05,
                                              'num_batches_per_epoch': 50,
                                              'patience': 10,
                                              'weight_decay': 1e-08},
                             'epochs': 100,
                             'learning_rate': 0.001,
                             'num_batches_per_epoch': 50,
                             'num_sampl

In [10]:

for x in [ 'model_pars', 'data_pars', 'compute_pars', 'out_pars' ] :
  globals()[x] = jpars['deepar'][x]
print( model_pars, data_pars, compute_pars, out_pars)



{'model_uri': 'model_gluon.gluonts_model', 'model_name': 'deepar', 'model_pars': {'prediction_length': 14, 'freq': 'D', 'distr_output': 'gluonts.distribution.student_t.StudentTOutput', 'use_feat_static_cat': True, 'use_feat_static_real': False, 'num_layers': 2, 'num_cells': 40, 'cell_type': 'lstm', 'dropout_rate': 0.1, 'scaling': True, 'num_parallel_samples': 100, 'cardinality': [4227]}} {'train': True, 'dt_source': '', 'data_type': 'gluonts', 'use_feat_static_cat': True, 'use_feat_static_real': False, 'submission': False, 'dataset_name': 'm4_daily', 'data_path': 'gluonts_data/m4_dataset/m4_daily', 'single_pred_length': 28, 'freq': '1D', 'start': '1750-01-01 00:00:00', 'startdate': '1750-01-01', 'col_date': 'timestamp', 'col_ytarget': ['target'], 'num_series': 1, 'cols_cat': [], 'cols_num': []} {'num_samples': 100, 'learning_rate': 0.001, 'epochs': 100, 'num_batches_per_epoch': 50, 'batch_size': 32, 'compute_pars': {'batch_size': 32, 'clip_gradient': 100, 'epochs': 100, 'init': 'xavier

In [11]:
### https://github.com/arita37/mlmodels/blob/dev/mlmodels/model_gluon/gluonts_model.py

#### Setup Model from repo
from mlmodels.models import module_load
#module         = module_load( model_uri)
#model          = module.Model(model_pars, data_pars, compute_pars) 


#### Local load model
import gluonts_model as module
# import imp
# imp.reload(module)


model = module.Model(model_pars=model_pars, data_pars=data_pars, compute_pars=compute_pars)
print(model)

INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


None
[4227]


In [0]:
#### Fit          >>>>USE module.fit
model = module.fit(model=model,data_pars= data_pars,compute_pars= compute_pars, out_pars=out_pars)           #### fit model

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/50 [00:00<?, ?it/s]INFO:numexpr.utils:NumExpr defaulting to 2 threads.


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


INFO:root:Number of parameters in DeepARTrainingNetwork: 245073
100%|██████████| 50/50 [00:09<00:00,  5.16it/s, avg_epoch_loss=7.5]
INFO:root:Epoch[0] Elapsed time 9.689 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.495181
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.99it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[1] Elapsed time 10.033 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.617773
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.00it/s, avg_epoch_loss=6.66]
INFO:root:Epoch[2] Elapsed time 9.998 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.657180
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.23it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[3] Elapsed time 9.573 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.444894
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.18it/s, avg_epoch_loss=6.55]
INF

In [0]:
y_pred = module.predict(model=model,data_pars= data_pars,compute_pars= compute_pars, out_pars=out_pars) 
 

In [0]:


metrics_val = module.fit_metrics(y_pred, data_pars, compute_pars, out_pars)   #### Check fit metrics
print(metrics_val)




In [0]:
!ls

In [0]:
module.plot_prob_forecasts(y_pred)

In [0]:
#### Save/Load
module.save(model, save_pars = { 'path': out_pars['path'] +"/model/"})

model2 = module.load(load_pars = { 'path': out_pars['path'] +"/model/"})



In [0]:
!ls 'kaggle_data'